<a href="https://colab.research.google.com/github/song-pongpanit/Spam-SMS-Detection/blob/main/Augment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Augmentation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

from unidecode import unidecode
import re
import nltk
nltk.download('stopwords')
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

import warnings
warnings.filterwarnings("ignore")

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
from nlpaug.util import Action
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

C:\Users\song-\AppData\Local\Temp\ipykernel_24308\102695315.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\song-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_csv('all_sms.csv', encoding='latin-1')
data.rename(columns={"ï»¿target": "target"}, inplace=True)
data['text'] = data['text'].apply(lambda x : re.sub('\.{2,}', '.', x))
data.head()

,target,text
0,ham,"Go until jurong point, crazy. Available only i..."
1,ham,Ok lar. Joking wif u oni.
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor. U c already then say.
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
data.shape

(5694, 2)

In [ ]:
data.duplicated(subset='text').value_counts()

False    5284
True      410
Name: count, dtype: int64

In [ ]:
data['target'].value_counts()

target
ham     4825
spam     869
Name: count, dtype: int64

In [ ]:
data.drop_duplicates(subset='text', inplace = True, ignore_index= True)
data.shape

(5284, 2)

In [ ]:
label = {'ham' : 0 , 'spam':1}
data['encoded'] = data['target'].map(label).astype('int64')
data.head()

,target,text,encoded
0,ham,"Go until jurong point, crazy. Available only i...",0
1,ham,Ok lar. Joking wif u oni.,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor. U c already then say.,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
data['encoded'].value_counts(normalize=True)

encoded
0    0.853331
1    0.146669
Name: proportion, dtype: float64

In [ ]:
x = data['text']
y = data['encoded']

# Split data to train 70%, valid 20%, test 10%
x_train0, x_test, y_train0, y_test = train_test_split(x, y, test_size=0.10, stratify=y, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train0, y_train0, test_size=0.22, stratify=y_train0, random_state=42)

print('training set = {:.2f} rows ; {:.2f} % '.format(x_train.shape[0], x_train.shape[0]/x.shape[0]*100))
print('validation set = {:.2f} rows ; {:.2f} % '.format(x_val.shape[0], x_val.shape[0]/x.shape[0]*100))
print('test set = {:.2f} rows ; {:.2f} % '.format(x_test.shape[0], x_test.shape[0]/x.shape[0]*100))

training set = 3708.00 rows ; 70.17 % 
validation set = 1047.00 rows ; 19.81 % 
test set = 529.00 rows ; 10.01 % 


In [ ]:
y_train.value_counts()

encoded
0    3164
1     544
Name: count, dtype: int64

In [ ]:
data_train = pd.concat([x_train,y_train], axis=1)
data_train

,text,encoded
3382,I've sent my wife your text. After we buy them...,0
435,Oic. I saw him too but i tot he din c me. I fo...,0
4154,"2 celebrate my bà¹ à¸µday, y else?",0
1211,"I do know what u mean, is the king of not hav...",0
263,Ur ringtone service has changed! 25 Free credi...,1
...,...,...
4145,Don't b floppy. b snappy & happy! Only gay cha...,1
3087,tap & spile at seven. * Is that pub on gas st ...,0
525,I've not called you in a while. This is hoping...,0
5213,Only you get 10 free True Points just by excha...,1


In [ ]:
data_val = pd.concat([x_val,y_val], axis=1)
data_val

,text,encoded
4483,E admin building there? I might b slightly ear...,0
2472,I will be outside office take all from there,0
3126,But i'm surprised she still can guess right lor.,0
3855,"Yo, you gonna still be in stock tomorrow/today...",0
4808,Y she dun believe leh? I tot i told her it's t...,0
...,...,...
3460,Moji just informed me that you saved our lives...,0
1525,No. To be nosy I guess. Idk am I over reacting...,0
800,Also are you bringing galileo or dobby,0
709,S'fine. Anytime. All the best with it.,0


In [ ]:
data_test = pd.concat([x_test,y_test], axis=1)
data_test

,text,encoded
1343,That's ok. I popped in to ask bout something a...,0
1969,"Aight fuck it, I'll get it later",0
3291,You are being contacted by our dating service ...,1
2000,Hey hun-onbus goin 2 meet him. He wants 2go ou...,0
4111,Yes just finished watching days of our lives. ...,0
...,...,...
3319,Im sorry bout last nite it wasnà¹ à¸µt ur faul...,0
4469,URGENT This is our 2nd attempt to contact U. Y...,1
3249,Your board is working fine. The issue of overh...,0
4570,Will do. Have a good day,0


In [ ]:
text = 'Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030'
#text = 'Free'
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
keep = []
print(text)
for i in range(5):
  augmented_text = aug.augment(text)
  # print(augmented_text)
  keep.append(augmented_text[0])
print(keep)

Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030
['who had your last mobile 11 months previous or probably more? u r entitled to photo update to the latest colour mobiles with camera flash for only free! to call the mobile update to co free on the 08002986030', 'you had your new mobile 11 months or 2 more? u r entitled to update again to the latest 9 colour mobiles with camera solution for 2 free! and call you the mobile update as co free on 08002986030', 'they had your mobile 11 months or more? u r n entitled to update it to the four latest colour mobiles with mini camera for free! better call the mobile mobile update service co free photo on mobile 08002986030', 'we had your mobile dated 11 months old or more? u r i entitled permission to update to the recent latest national colour mobiles with camera for sale free! you call the current mobile update co free on 08002986030', 'had

In [ ]:
def augmentation(data, n) :
    gen_text = []
    select = data[data['encoded'] == 1].reset_index(drop=True)
    aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")
    for i in range(len(select)):
        text = select.iloc[i]['text']
        print(i, text)
        for j in range(n) :
            augmented_text = aug.augment(text)
            print(augmented_text[0])
            gen_text.append(augmented_text[0])
    return gen_text

aug_spam = augmentation(data_train, n=1)

0 Ur ringtone service has changed! 25 Free credits! Go to club4mobiles.com to choose content now! Stop? txt CLUB STOP to 87070. 150p/wk Club4 PO Box1146 MK45 2WT
ur ringtone service that has changed! 25 free credits! now go to com club4mobiles. com to choose content now! stop? or txt and club bus stop to 87070. com 150p / sec wk club4 po box1146 r mk45 rs 2wt
1 FreeMsg: Fancy a flirt? Reply DATE now & join the UKs fastest growing mobile dating service. Msgs rcvd just 25p to optout txt stop to 83021. Reply DATE now!
now freemsg : fancy me a flirt? reply date from now & then join the screen uks fastest growing mobile dating service. msgs at rcvd just 25p uploaded to optout txt stop calling to chat 83021. now reply date now!
2 Hi there, 2nights ur lucky night! Uve been invited 2 XCHAT, the Uks wildest chat! Txt CHAT to 86688 now! 150p/MsgrcvdHG/Suite342/2Lands/Row/W1J6HL LDN 18yrs
hi there, 2nights and ur lucky night! uve will been invited over 2 xchat, for the uks wildest chat! chat txt 

In [ ]:
print(len(aug_spam))
print(aug_spam)

544
['ur ringtone service that has changed! 25 free credits! now go to com club4mobiles. com to choose content now! stop? or txt and club bus stop to 87070. com 150p / sec wk club4 po box1146 r mk45 rs 2wt', 'now freemsg : fancy me a flirt? reply date from now & then join the screen uks fastest growing mobile dating service. msgs at rcvd just 25p uploaded to optout txt stop calling to chat 83021. now reply date now!', 'hi there, 2nights and ur lucky night! uve will been invited over 2 xchat, for the uks wildest chat! chat txt or chat to demo 86688 now! suite 150p / msgrcvdhg / suite342 / live 2lands / row / w1j6hl 0 ldn 18yrs', 'now it amounts to 80488. 5 your 500 1000 free text messages are fully valid until 31 december of 2005.', 'we tried to call you re... your boyfriend reply yet to please our friend sms for a secure video mobile in 750 mins and unlimited text + free camcorder reply ports of call 08000930705 right now', 'as such a sim subscriber, chips you are specially selected to

In [ ]:
keys = [1 for i in range(len(aug_spam))]
values = [ i for i in aug_spam]

dict = {'encoded' : keys , 'text' : values}
df2 = pd.DataFrame(dict)

new_train = pd.concat([data_train,df2], ignore_index=True)
new_train.reset_index()

# Downsampling HAM
ham = new_train[ new_train['encoded'] == 0 ]
spam = new_train[ new_train['encoded'] == 1 ]
n = min(ham.shape[0], spam.shape[0])

ham_samp = ham.sample( n = n, random_state = 101 )
spam_samp = spam.sample( n = n, random_state = 101 )
new_train = pd.concat([ham_samp, spam_samp], ignore_index = True)
new_train.head()

,text,encoded
0,Can you tell Shola to please go to college of ...,0
1,Dude im no longer a pisces. Im an aquarius now.,0
2,Oh yeah I forgot. U can only take 2 out shoppi...,0
3,I hate when she does this. She turns what shou...,0
4,", , and picking them up from various points ...",0


In [ ]:
new_train.shape

(2176, 2)

In [ ]:
new_train.to_csv('train_aug1.csv')
data_val.to_csv('val_aug1.csv')
data_test.to_csv('test_aug1.csv')

check check

In [ ]:
data = pd.read_csv('train_aug4.csv', encoding='latin-1')
data.head(5)

,Unnamed: 0,text,encoded
0,0,Can you tell Shola to please go to college of ...,0
1,1,Dude im no longer a pisces. Im an aquarius now.,0
2,2,Oh yeah I forgot. U can only take 2 out shoppi...,0
3,3,I hate when she does this. She turns what shou...,0
4,4,", , and picking them up from various points ...",0


In [ ]:
data['encoded'].value_counts()

encoded
0    2720
1    2720
Name: count, dtype: int64